In [ ]:
%pip install transformers 
%pip install sklearn
%pip install datasets
%pip install SentencePiece

In [ ]:
from transformers import XLNetForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric,Dataset
import torch
import numpy as np
from datasets import load_dataset
from utils import PrepareCorpus,AssessData
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
import json
from sklearn.model_selection import KFold

20 NEWSGROUP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
parentdir = "/content/drive/MyDrive/ML/Datasets/20News"

train =parentdir+"/20news-bydate"+"/20news-bydate-train"
test = parentdir+"/20news-bydate"+ "/20news-bydate-test"

In [ ]:
xlnet_tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
xlnet_model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels = 20)

In [ ]:
id2labels = {'talk.politics.mideast': 0, 'rec.autos': 1, 'comp.sys.mac.hardware': 2, 'alt.atheism': 3, 'rec.sport.baseball': 4, 'comp.os.ms-windows.misc': 5, 'rec.sport.hockey': 6, 'sci.crypt': 7, 'sci.med': 8, 'talk.politics.misc': 9, 'rec.motorcycles': 10, 'comp.windows.x': 11, 'comp.graphics': 12, 'comp.sys.ibm.pc.hardware': 13, 'sci.electronics': 14, 'talk.politics.guns': 15, 'sci.space': 16, 'soc.religion.christian': 17, 'misc.forsale': 18, 'talk.religion.misc': 19}
id2classes = {'talk.politics.mideast': 0, 'rec.autos': 4, 'comp.sys.mac.hardware': 2, 'alt.atheism': 5, 'rec.sport.baseball': 1, 'comp.os.ms-windows.misc': 2, 'rec.sport.hockey': 1, 'sci.crypt': 3, 'sci.med': 3, 'talk.politics.misc': 0, 'rec.motorcycles': 4, 'comp.windows.x': 2, 'comp.graphics': 2, 'comp.sys.ibm.pc.hardware': 2, 'sci.electronics': 3, 'talk.politics.guns': 0, 'sci.space': 3, 'soc.religion.christian': 5, 'misc.forsale': 6, 'talk.religion.misc': 5}

number = range(50,150,50)
_per_segment = range(200,400,100)

overlap = {"side":"both", "number": 50}

f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

def tokenize(batch):
    return xlnet_tokenizer(batch['text'], truncation=True,padding = True)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):
  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.00001,0.00005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)}

def xlnet_init():
  return XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels = 20)

def id_2_labels(x,adict):
  """converts labels/classes into a number using a dictionary"""
  return adict[x]

def _indexing(alist):
  id = 0; indexed = {}
  for x in alist:
    for y in x:
      id +=1
      indexed[id] = y
  return indexed
  

In [ ]:
news_group_train = PrepareCorpus(train)
news_group_test = PrepareCorpus(test)

In [ ]:
news_group_test = news_group_test._prep()       #Keys are labels, values are texts
news_group_train = news_group_train._prep()

SELECTING TWO CLASSES

In [ ]:
#Selecting politics.mideast, politics.misc and sport.baseball, sport.hockey
labels = ['talk.politics.mideast', 'talk.politics.misc', 'talk.politics.guns', 'rec.sport.baseball','rec.sport.hockey']

temp_train = []; temp_test = []
temp_news_group_train = {}; temp_news_group_test = {}

for item in labels:
  temp_news_group_train[item] = news_group_train.get(item)
  temp_news_group_test[item] = news_group_test.get(item)

temp_train = _indexing(temp_news_group_train.values())
temp_test =  _indexing(temp_news_group_test.values())

temp_train_corpus = AssessData(temp_train, temp_news_group_train)
temp_test_corpus = AssessData(temp_test, temp_news_group_test)

In [ ]:
type(temp_train_corpus)

utils.AssessData

In [ ]:
temp_train_index_to_label = temp_train_corpus._index_to_label() 
temp_test_index_to_label = temp_test_corpus._index_to_label()

In [ ]:
temp_train_index_to_label = {i:id_2_labels(x,id2classes) for i,x in temp_train_index_to_label.items()}
temp_test_index_to_label = {i:id_2_labels(x,id2classes) for i,x in temp_test_index_to_label.items()}

In [ ]:
set(temp_train_index_to_label.values())

{0, 1}

In [ ]:
temp_train_set = temp_train_corpus._chunk(200, overlap=overlap)
temp_train_index, temp_train_text = zip(*temp_train_set)

temp_test_set = temp_test_corpus._chunk(200, overlap=overlap)
test_index, test_text = zip(*temp_test_set)

In [ ]:
frac_train_set = {"train" : {"text": temp_train.values(), 'doc_id': temp_train_index_to_label.keys(), 'labels': temp_train_index_to_label.values()}}
frac_test_set =  {"test" : {"text": temp_test.values(), 'doc_id': temp_test_index_to_label.keys(), 'labels': temp_test_index_to_label.values()}}

In [ ]:
frac_train_set = Dataset.from_dict(frac_train_set['train'])
frac_test_set = Dataset.from_dict(frac_test_set['test'])

In [ ]:
frac_train_set = frac_train_set.map(tokenize)
frac_test_set = frac_test_set.map(tokenize)

frac_train_set = frac_train_set.shuffle(seed = 24)
frac_test_set = frac_test_set.shuffle(seed = 24)

  0%|          | 0/2772 [00:00<?, ?ex/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/1846 [00:00<?, ?ex/s]

In [ ]:
frac_test_set

Dataset({
    features: ['text', 'doc_id', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1846
})

In [ ]:
fold = KFold(n_splits = 5)
this_set = fold.split(frac_train_set['text'])

frac_fold_train = []; frac_fold_eval = []

for i,j in this_set:
  frac_fold_train.append(frac_train_set.select(i))
  frac_fold_eval.append(frac_train_set.select(j))

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=xlnet_tokenizer, max_length= 300)

#default training arguments 
training_args = TrainingArguments(output_dir="./frac_results", learning_rate=2e-5, per_device_train_batch_size=16, per_device_eval_batch_size=16, num_train_epochs=5,weight_decay=0.01)
results = []

#fine-tune
trainer = Trainer(model = model, 
                args = training_args,
                train_dataset= frac_fold_train[0],
                eval_dataset= frac_fold_eval[0],
                tokenizer= tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer.train()

In [ ]:
trainer_1 = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[1],
                eval_dataset= frac_fold_eval[1],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer_1.train()

In [ ]:
trainer_1.evaluate()

In [ ]:
trainer_2 = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[2],
                eval_dataset= frac_fold_eval[2],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer_2.train()

In [ ]:
trainer_2.evaluate()

In [ ]:
trainer_3 = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[3],
                eval_dataset= frac_fold_eval[3],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer_3.train()

In [ ]:
trainer_3.evaluate()

In [ ]:
trainer_4 = Trainer(model = bert_model, 
                args = training_args,
                train_dataset= frac_fold_train[3],
                eval_dataset= frac_fold_eval[3],
                tokenizer= bert_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer_4.train()

In [ ]:
trainer_4.evaluate()

In [ ]:
trainer.predict(frac_test_set)

FULL TRAIN AND TEST DATA

In [ ]:
#Full train and test data

train_list_of_strings = _indexing(news_group_train.values())
test_list_of_strings = _indexing(news_group_test.values())

train_corpus = AssessData(train_list_of_strings, news_group_train)
test_corpus = AssessData(test_list_of_strings, news_group_test)

In [ ]:
train_index_to_label = train_corpus._index_to_label() 
test_index_to_label = test_corpus._index_to_label()

In [ ]:
train_list_of_strings

In [ ]:
len(set(train_index_to_label.keys()))

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
test_index = {a:x-1 for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [ ]:
train_chunked_label = {i:id_2_labels(x,train_index_to_label) for i,x in train_index.items()}
test_chunked_label = {i:id_2_labels(x,train_index_to_label) for i,x in test_index.items()}

In [ ]:
len(train_index.values()), len(train_text.values()), len(train_chunked_label.values())

In [ ]:
train = [xlnet_tokenizer(x) for x in train_list_of_strings.values()]

In [ ]:
new_train = {i:x for i,x in enumerate(train)}

In [ ]:
test = [xlnet_tokenizer(x) for x in test_list_of_strings.values()]

In [ ]:
new_test = {i:x for i,x in enumerate(test)}

In [ ]:
temp_train = pd.DataFrame(new_train.values())
temp_test = pd.DataFrame(new_test.values())

In [ ]:
train_set = Dataset.from_pandas(temp_train)
test_set = Dataset.from_pandas(temp_test)

In [ ]:
train_set.add_column("doc_id", new_train.keys())
test_set.add_column('doc_id', new_test.keys())

In [ ]:
train = train.shuffle(seed = 30)
test = test.shuffle(seed = 30)

In [ ]:
from re import template
from sklearn.model_selection import KFold

fold = KFold(n_splits = 5)
this_set = fold.split(train_set['input_ids'])

fold_train = []
fold_eval = []

for i,j in this_set:
  fold_train.append(train_set.select(i))
  fold_eval.append(train_set.select(j))

In [ ]:
%pip install optuna

In [ ]:
training_args = TrainingArguments(output_dir="./results_temp", learning_rate=2e-5, per_device_train_batch_size=32, per_device_eval_batch_size=32, num_train_epochs=5,weight_decay=0.01)
results = []

max_input_length = 300

data_collator = DataCollatorWithPadding(tokenizer=xlnet_tokenizer, padding = 'longest')
xlnet_model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels = 20)

result = {}

#fine-tune

trainer = Trainer(model = xlnet_model, 
                args = training_args,
                train_dataset= fold_train[0],
                eval_dataset= fold_eval[0],
                tokenizer= xlnet_tokenizer,
                data_collator=data_collator,
                compute_metrics= compute_metrics)

trainer.train()


In [ ]:
result 

{KFold(n_splits=5, random_state=None, shuffle=False): {'epoch': 5.0,
  'eval_f1': {'f1': 0.934695986240775},
  'eval_loss': 0.17530567944049835,
  'eval_precision': {'precision': 0.9457526149958522},
  'eval_recall': {'recall': 0.9325851363338347},
  'eval_runtime': 301.1003,
  'eval_samples_per_second': 86.114,
  'eval_steps_per_second': 2.693}}

In [ ]:
with open("7_classes.json", "w") as ins:
  json.dump(result, ins)

In [ ]:
import json
with open ("eval_32_32", "w") as out: 
  json.dump(eval_32_32, out)